In [1]:
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
import numpy as np

In [21]:
def processing(filename):
    doc = ""
    length = 0
    with open(filename) as f:
        doc = f.read()
    #print(doc)
    sent_tokens = sent_tokenize(doc)
    #print('Number of lines = ', len(sent_tokens))

    # Tokenize words:
    word_tokens = [[w.lower() for w in word_tokenize(text)] for text in sent_tokens]
    #print(word_tokens)

    # Create the dictionary using gensim
    dictionary = gensim.corpora.Dictionary(word_tokens)
    #print(dictionary.token2id)

    # Create a corpus/vectorization:
    corpus = [dictionary.doc2bow(token) for token in word_tokens]
    #print(corpus) # will give (vector_number_given_in_the_step_above, frequency_of_that_word_in_the_sentence) format
    
    return (word_tokens, dictionary, corpus)

In [22]:
(word_tokens, dictionary, corpus_file1) = processing('file1.txt')
print(corpus_file1)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2)], [(0, 1), (3, 1), (5, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)], [(0, 1), (3, 1), (5, 1), (16, 1), (17, 1)]]


In [23]:
# Calculate the frequency (TFID Term frequency)
tf_idf = gensim.models.TfidfModel(corpus_file1)
#print(tf_idf[corpus_file1], '\n')
for doc in tf_idf[corpus_file1]:
    #print(doc, '\n')
    print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc], '\n')

[['fourth', 0.47], ['from', 0.47], ['mars', 0.47], ['sun', 0.47], ['the', 0.35]] 

[['the', 0.25], ['after', 0.34], ['in', 0.34], ['it', 0.34], ['mercury', 0.34], ['second', 0.34], ['smallest', 0.34], ['solar', 0.34], ['system', 0.34]] 

[['saturn', 0.71], ['yellow', 0.71]] 



In [24]:
# Similarity measure
sims = gensim.similarities.Similarity('', tf_idf[corpus_file1], num_features=len(dictionary)) 
# '' will have the working directory

In [30]:
(q_word_tokens, dictionary2, corpus_query_doc) = processing('file2.txt')
print(q_word_tokens)
for line in q_word_tokens:
    q_bow = dictionary.doc2bow(line)
print(q_bow)

[['saturn', 'is', 'sixth', 'planet', 'from', 'sun', '.']]
[(0, 1), (2, 1), (3, 1), (5, 1), (6, 1), (16, 1)]


In [31]:
# Perform a similarity query against the original corpus:
query_tf_idf = tf_idf[q_bow]
print(query_tf_idf)
print('Comparing results:', sims[query_tf_idf])

# Adding the similarities
import numpy as np
sum_of_sims =(np.sum(sims[query_tf_idf], dtype=np.float32))
print(sum_of_sims)

# Getting the similarity percentage:
percentage_of_similarity = round(float((sum_of_sims / len(word_tokens)) * 100))
print(f'Average similarity float: {float(sum_of_sims / len(word_tokens))}')
print(f'Average similarity percentage: {float(sum_of_sims / len(word_tokens)) * 100}')
print(f'Average similarity rounded percentage: {percentage_of_similarity}')

[(2, 0.5773502691896258), (6, 0.5773502691896258), (16, 0.5773502691896258)]
Comparing results: [0.5416385  0.         0.40824828]
0.9498868
Average similarity float: 0.31662893295288086
Average similarity percentage: 31.662893295288086
Average similarity rounded percentage: 32
